In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


### Import the QmioBackend code

In [ ]:
from qmiotools.integrations.qiskitqmio import QmioBackend
from qmio import QmioRuntimeService

In [ ]:
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit import transpile
from qiskit.visualization import plot_gate_map
from qiskit import qasm3

In [ ]:
c=QuantumCircuit(4)
a=Parameter("a")
c.h(0)
c.cry(a, 0,1)
c.measure_all()
c.draw(output="mpl")

In [ ]:
c2=QuantumCircuit(4)
a=Parameter("a")
b=Parameter("b")
c2.h(0)
c2.ry(a, 0)
c2.ry(b, 1)
c2.cx(0,1)
c2.measure_all()
c2.draw(output="mpl")

In [ ]:
qmio=QmioBackend()

In [ ]:
d=transpile(c,qmio)
print(d.depth())
d.draw(output="mpl")

In [ ]:
d=transpile(c2,qmio)
print(d.depth())
d.draw(output="mpl")

In [ ]:
from qmio.backends import QPUBackend, _config_build
def run2(self, circuit, shots, repetition_period=400e-6):
    if not self.client:
        raise RuntimeError("Not connected to the server")
    print("LLamando a run2 con %f"%repetition_period)

    config = _config_build(shots,repetition_period=repetition_period)
    #print(config)
    job = (circuit, config)
    self.client._send(job)
    result = self.client._await_results()
    return result

    
class QmioRuntimeService():
    QPUBackend.run=run2
    def backend(self, name):
        if name == "qpu":
            return QPUBackend()
        # if name == "qulacs":
        #     pass
        else:
            raise ValueError(f"Backend unknown: {name}")

### Create an instance of the QMIO backend, loading he last calibration from the directory given by the QMIO_CALIBRATIONS environ

In [ ]:
from typing import Union, List
#from qiskit.circuit import Schedule
from qiskit.pulse import Schedule
from qiskit.providers import QubitProperties, BackendV2, Provider, Options, Job, JobStatus

from qmiotools.integrations.qiskitqmio import QmioJob

from datetime import date
import uuid
from dataclasses import dataclass
from qiskit.result import Result, Counts 

class QPUException(Exception):
    pass

def run2(self, circuit, shots, repetition_period=400e-6):
    if not self.client:
        raise RuntimeError("Not connected to the server")
    print("LLamando a run2 con %f"%repetition_period)

    config = _config_build(shots,repetition_period=repetition_period)
    #print(config)
    job = (circuit, config)
    self.client._send(job)
    result = self.client._await_results()
    return result
@dataclass
class QasmCircuit():
    circuit=None
    name=None
    
class QmioRuntimeService():
    QPUBackend.run=run2
    def backend(self, name):
        if name == "qpu":
            return QPUBackend()
        # if name == "qulacs":
        #     pass
        else:
            raise ValueError(f"Backend unknown: {name}")
def run_experimental(self, run_input: Union[Union[QuantumCircuit,Schedule, str],List[Union[QuantumCircuit,Schedule,str]]], **options):
    """Run on the backend.



    Args:
        run_input (QuantumCircuit or Schedule or Str - a QASM string - or list): An
            individual or a list of :class:`.QuantumCircuit`,
            or :class:`~qiskit.pulse.Schedule` or a string with a QASM 3.0 objects to
            run on the backend.
        options: Any kwarg options to pass to the backend for running the
            config. If a key is also present in the options
            attribute/object then the expectation is that the value
            specified will be used instead of what's set in the options
            object.

    Returns:
        Job: The job object for the run
    """

    if isinstance(options,Options):
        shots=options.get("shots",default=self._options.get("shots"))
        memory=options.get("memory",default=self._options.get("shots"))
        repetition_period=options.get("repetition_period",default=self._options.get("repetition_period"))
    else:
        if "shots" in options:
            shots=options["shots"]
        else:
            shots=self._options.get("shots")

        if "memory" in options:
            shots=options["memory"]
        else:
            memory=self._options.get("memory")

        if "repetition_period" in options:
            repetition_period=options["repetition_period"]
        else:
            repetition_period=self._options.get("repetition_period")


    if isinstance(run_input,QuantumCircuit) or isinstance(run_input,Schedule) or isinstance(run_input,str):
        circuits=[run_input]
    else:
        circuits=run_input

    if shots*len(circuits) > self.max_shots:
        raise RuntimeError("Total number of shots %d larger than capacity %d"%(shots,self.max_shots))


    service = QmioRuntimeService()
    #print(service)
    job_id=uuid.uuid4()
    ExpResult=[]
    with service.backend(name="qpu") as backend:
        for c in circuits:
            if isinstance(c,QuantumCircuit) or isinstance(c,Schedule):
                qasm=qasm3.dumps(c, basis_gates=self.operation_names).replace("\n","")
                print(qasm)
            else:
                qasm=c
            remain_shots=shots
            ExpDict={}
            backend.run2=run2

            while (remain_shots > 0):
                results = backend.run(circuit=qasm, shots=min(self._max_shots,remain_shots),repetition_period=repetition_period)
                #print("Results:",results)
                if "Exception" in results:
                    raise QPUException(results["Exception"])
                try:
                    r=results["results"][list(results["results"].keys())[0]]
                except:
                    raise QPUException("QPU does not return results")
                for k in r:
                    if not memory:
                        key=hex(int(k[::-1],base=2))
                        ExpDict[key]=ExpDict[key]+r[k] if key in ExpDict else r[k]
                    else:
                        raise RuntimeError("Binary for the next version")
                remain_shots=remain_shots-self._max_shots
            metadata=None
            #print(ExpDict)
            if "execution_metrics" in results:
                metadata=results["execution_metrics"] 
            """ExpResult.append(ExperimentResult(
                shots,
                success=True,
                data=Counts(ExpDict),
                meas_level=MeasLevel.CLASSIFIED,
                status=None,
                seed=None,
                meas_return=None,
                header=QobjExperimentHeader.from_dict({"name":c.name,"metadata":metadata})))"""
            if isinstance(c,str):
                c_copy=c
                c=QasmCircuit()
                c.circuit=c_copy
                c.name="QASM"
                print(c)
                
            dd={
                'shots': shots,
                'success': True,
                'data': {

                    'counts': ExpDict,
                    'metadata': None,
                },
                'header': {'name': c.name},

                }
            ExpResult.append(dd)

    result_dict = {
        'backend_name': self._name,
        'backend_version': self._version,
        'qobj_id': None,
        'job_id': job_id,
        'success': True,
        'results': ExpResult,
        'date': date.today().isoformat(),

    }
    #print(ExpResult)
    results=Result.from_dict(result_dict)
    
    job=QmioJob(backend=self,job_id=uuid.uuid4(), jobstatus=JobStatus.DONE, 
result=results)
    return job

QmioBackend.run=run_experimental
backend=QmioBackend()

In [ ]:
backend=QmioBackend()

In [ ]:
%matplotlib inline
c=QuantumCircuit(2)
c.h(0)
c.cx(0,1)
c.measure_all()
c.draw(output="mpl")

In [ ]:
d=c.copy()
d.draw(output="mpl")

In [ ]:
qc_basis = transpile(c, backend)
#qc_basis.draw(output="mpl")

In [ ]:
qd_basis = transpile(d, backend)
#qd_basis.draw(output="mpl")

In [ ]:
c=QuantumCircuit(2)
c.measure_all()
c.delay(1e-6,unit="s")
c.name="ONLY INIT"
qasm3.dumps(c)

In [ ]:
shots=200
repetition_period=0.1
job=backend.run([c,d], shots=shots, repetition_period=repetition_period)

In [ ]:
c=QuantumCircuit(2)
c.measure_all()
#c.delay(1e-6,unit="s")
c.name="ONLY INIT"
d=QuantumCircuit(2)
d.x(0)
d.measure_all()
d.name="X in cubit 0"
c=transpile(c,backend,optimization_level=3)
d=transpile(d,backend,optimization_level=2)
c.draw()
c="""
OPENQASM 3.0;include "stdgates.inc";bit[2] meas;barrier $0, $1;meas[0] = measure $0;meas[1] = measure $1;
"""
d="""
OPENQASM 3.0;include "stdgates.inc";bit[2] meas;sx $0;sx $0;barrier $0, $1;meas[0] = measure $0;meas[1] = measure $1;
"""
e="""
OPENQASM 3.0;include "stdgates.inc";bit[2] meas;sx $1;sx $1;barrier $0, $1;meas[0] = measure $0;meas[1] = measure $1;
"""

print(c)

# These cells only work if executed in a node with connection to the QPU

In [ ]:
shots=200
repetition_period=0.1
job=backend.run([c,d,e], shots=shots, repetition_period=repetition_period)
#print(job.result())
print(job.result().get_counts())
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Repetition_period=%f"%repetition_period,legend=["ONLY_INIT","X(0)","X(1)"])

In [ ]:
repetition_period=0.01
job=backend.run([c,d,e], shots=shots, repetition_period=repetition_period)
#print(job.result())
print(job.result().get_counts())
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Repetition_period=%f"%repetition_period,legend=["ONLY_INIT","X(0)","X(1)"])

In [ ]:
repetition_period=0.001
job=backend.run([c,d,e], shots=shots, repetition_period=repetition_period)
#print(job.result())
print(job.result().get_counts())
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Repetition_period=%f"%repetition_period,legend=["ONLY_INIT","X(0)","X(1)"])

In [ ]:
repetition_period=0.0001
job=backend.run([c,d,e], shots=shots, repetition_period=repetition_period)
#print(job.result())
print(job.result().get_counts())
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Repetition_period=%f"%repetition_period,legend=["ONLY_INIT","X(0)","X(1)"])

In [ ]:
repetition_period=0.00001
job=backend.run([c,d,e], shots=shots, repetition_period=repetition_period)
#print(job.result())
print(job.result().get_counts())
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Repetition_period=%f"%repetition_period,legend=["ONLY_INIT","X(0)","X(1)"])

In [ ]:
repetition_period=0
job=backend.run([c,d,e], shots=shots)
#print(job.result())
print(job.result().get_counts())
from qiskit.visualization import plot_histogram
plot_histogram(job.result().get_counts(), title="Repetition_period=%f"%repetition_period,legend=["ONLY_INIT","X(0)","X(1)"])

In [ ]:
jobs=[]
shots=1000
for i in range(10):
    repetition_period=100e-6*(i+1)
    job=backend.run([c,d,e], shots=shots, repetition_period=repetition_period)
    jobs.append(job)
    print(repetition_period,job.result().get_counts())
    #plot_histogram(job.result().get_counts(), title="Repetition_period=%f"%repetition_period,legend=["ONLY_INIT","X(0)","X(1)"])

In [ ]:
d="""
OPENQASM 3.0;include "stdgates.inc";bit[1] meas;sx $0;sx $0;delay[{0}us] $0;barrier $0;meas[0] = measure $0;
"""
d.format(10)

In [ ]:
import logging
type(logging.INFO)
logger = logging.getLogger("QmioBackend")

logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')



import sys
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)
logger.info("kkk")

In [ ]:
import matplotlib.pyplot as plt
jobs=[]
counts=[]
shots=1000
Times=20
for i in range(Times):
    delay=(i)*10
    rund=d.format(delay)
    repetition_period=400e-6*(i+1)
    job=backend.run(rund, shots=shots, repetition_period=500e-6)
    jobs.append(job)
    print(delay, i,job.result().get_counts())
    counts.append(job.result().get_counts()["1"])
plt.plot([j*10 for j in range(Times)], counts,"*")

In [4]:
d="""
OPENQASM 3.0;include "stdgates.inc";bit[1] meas;sx ${0};sx ${0};delay[{1}us] ${0};barrier ${0};meas[0] = measure ${0};
"""
d.format(2,10)

'\nOPENQASM 3.0;include "stdgates.inc";bit[1] meas;sx $2;sx $2;delay[10us] $2;barrier $2;meas[0] = measure $2;\n'

In [3]:
from qmiotools.integrations.qiskitqmio import QmioBackend
d="""
OPENQASM 3.0;include "stdgates.inc";bit[1] meas;sx $2;sx $2;delay[10us] $2;barrier $2;meas[0] = measure $2;
"""
print("Salida raw")
for i in ['binary_count', 'raw', 'binary', 'squash_binary_result_arrays']:
    print("\n\n\nSalida %s"%i)
    try:
        QmioBackend().run(d,res_format=i)
    except:
        pass

2024-05-30 17:46:24,664 - QmioBackend - INFO - Logging started:
2024-05-30 17:46:24,664 - QmioBackend - INFO - Logging started:
2024-05-30 17:46:24,668 - QmioBackend - INFO - Loading calibrations
2024-05-30 17:46:24,668 - QmioBackend - INFO - Loading calibrations
Importing calibrations from  /opt/cesga/qmio/hpc/calibrations/2024_05_24__17_16_00.json
len: 32


QmioException: Format Raw not in available formats:['binary_count', 'raw', 'binary', 'squash_binary_result_arrays']

In [ ]:
import matplotlib.pyplot as plt
jobs=[]
counts=[]
shots=1000
Times=20
qubit=0
for i in range(Times):
    delay=(i)*10
    rund=d.format(qubit,delay)
    repetition_period=400e-6*(i+1)
    job=backend.run(rund, shots=shots, repetition_period=500e-6)
    jobs.append(job)
    print(delay, i,job.result().get_counts())
    counts.append(job.result().get_counts()["1"])
plt.plot([j*10 for j in range(Times)], counts,"*")
plt.title("Qubit %d"%qubit)

In [ ]:
import matplotlib.pyplot as plt
jobs=[]
counts=[]
shots=1000
Times=20
qubit=1
for i in range(Times):
    delay=(i)*10
    rund=d.format(qubit,delay)
    print(rund)
    repetition_period=400e-6*(i+1)
    job=backend.run(rund, shots=shots, repetition_period=500e-6)
    jobs.append(job)
    print(delay, i,job.result().get_counts())
    counts.append(job.result().get_counts()["1"])
plt.plot([j*10 for j in range(Times)], counts,"*")
plt.title("Qubit %d"%qubit)

In [ ]:
import matplotlib.pyplot as plt
jobs=[]
counts=[]
shots=1000
Times=20
qubit=2
for i in range(Times):
    delay=(i)*10
    rund=d.format(qubit,delay)
    print(rund)
    repetition_period=400e-6*(i+1)
    job=backend.run(rund, shots=shots, repetition_period=500e-6)
    jobs.append(job)
    print(delay, i,job.result().get_counts())
    counts.append(job.result().get_counts()["1"])
plt.plot([j*10 for j in range(Times)], counts,"*")
plt.title("Qubit %d"%qubit)

In [ ]:
c=QuantumCircuit(2)
c.x(0)
c.delay(1, unit="us")
c.measure_all()
c.name="ONLY INIT"
qasm3.dumps(c)

In [ ]:
job=backend.run(c, shots=shots, repetition_period=repetition_period)